In [2]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate


llm = Ollama(model="llama3") # local model, open source

PFMEA_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "Based on the provided manufacturing process, please generate Process Failure Modes and Effects Analysis (PFMEA) in the following format:\n"
          "Here are some examples:\n"
          "placeholder here ..\n"
          "For each unique Process Step, please generate at least 5 distinct Potential Failure Modes and Potential Effect, along with corresponding Recommended Actions. Please include all steps mentioned in the input data and generate as much text as possible. The manufacturing process is as follows:\n\n"
     ),
    ("user", "{input}")
])

PFMEA_chain = PFMEA_prompt | llm 

PFMEA_result = PFMEA_chain.invoke({"input": text})

# 8-10 lines of code for running llama 3 model locally !!

In [4]:
import pandas as pd
import os
import docx2txt
from io import StringIO
import time


pd.set_option('display.max_colwidth', None)


def load_data(file_path):
    if file_path.endswith('.docx'):
        text = docx2txt.process(file_path)
    else:
        with open(file_path, 'r') as file:
            text = file.read()
    return text
    
paths = ["./data/Lathing_Process.docx"]
text = load_data(paths[0])
PFMEA_result = PFMEA_chain.invoke({"input": text})
print(PFMEA_result)

**Process Failure Modes and Effects Analysis (PFMEA)**

**Step 4.1: Preparation**

1. **Potential Failure Mode:** Lathe machine failure due to improper maintenance.
	* **Effect:** Delays in production, increased downtime, and potential for tool damage or breakage.
	* **Recommended Action:** Regularly inspect the lathe machine, perform routine maintenance, and keep records of all maintenance activities.
2. **Potential Failure Mode:** Inadequate personal protective equipment (PPE).
	* **Effect:** Operator injury or exposure to hazardous materials.
	* **Recommended Action:** Ensure PPE is always worn by operators, and conduct regular checks for wear and tear.
3. **Potential Failure Mode:** Insufficient information review.
	* **Effect:** Machining errors, tool damage, or part defects.
	* **Recommended Action:** Review lathe specifications and drawings thoroughly before starting the machining operation.
4. **Potential Failure Mode:** Inadequate cleaning of the work area.
	* **Effect:** Debr

In [ ]:


format_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "Please process input data into a list of json format, each json payload has the following keys, with value being one string for each row, not list, do not change content under each Process Step:\n"
          "placeholder here ..\n"
          "Please process entire input data into a list of json format, value for process step can repeat for multiple failure modes under same process step, do not change any input data, make sure you include all data from input, and make sure every json payloads has same keys. The output should be a list of json of the data payloads only. The given input is as follows:\n\n"
     ),
    ("user", "{input}")
])
format_chain = format_prompt | llm 


In [2]:
paths = ["./data/Lathing_Process.docx"]
text = load_data(paths[0])
PFMEA_result = PFMEA_chain.invoke({"input": text})
print(PFMEA_result)

Here is the Process Failure Modes and Effects Analysis (PFMEA) for the Lathe Operation Work Instruction:

**Preparation**

| Potential Failure Mode | Potential Effect | Recommended Actions |
| --- | --- | --- |
| Improper review of lathe specifications and drawings | Inaccurate machining parameters, leading to poor quality and potential rework | Verify lathe specifications and drawings before starting the operation |
| Unclean or obstructed work area | Hazardous working conditions, risk of accidents, and potential damage to equipment or materials | Ensure a clean and well-ventilated work area |

**Material Preparation**

| Potential Failure Mode | Potential Effect | Recommended Actions |
| --- | --- | --- |
| Insufficient cleaning of the workpiece | Presence of contaminants on the machined surface, affecting quality and performance | Ensure thorough cleaning of the workpiece before machining |
| Incorrect selection of workholding device | Misalignment or inadequate holding of the workp

In [4]:
final_result = format_chain.invoke({"input": PFMEA_result})
print(final_result)


Here is the processed input data in JSON format:

```
[
    {
        "Process Step": "Preparation",
        "Potential Failure Mode": "Improper review of lathe specifications and drawings",
        "Potential Effect": "Inaccurate machining parameters, leading to poor quality and potential rework",
        "Recommended Actions": "Verify lathe specifications and drawings before starting the operation"
    },
    {
        "Process Step": "Preparation",
        "Potential Failure Mode": "Unclean or obstructed work area",
        "Potential Effect": "Hazardous working conditions, risk of accidents, and potential damage to equipment or materials",
        "Recommended Actions": "Ensure a clean and well-ventilated work area"
    },
    {
        "Process Step": "Material Preparation",
        "Potential Failure Mode": "Insufficient cleaning of the workpiece",
        "Potential Effect": "Presence of contaminants on the machined surface, affecting quality and performance",
        "Recommend

In [5]:
df = parse_into_dataframe(final_result)
df.head()

,Process Step,Potential Failure Mode,Potential Effect,Recommended Actions
0,Preparation,Improper review of lathe specifications and drawings,"Inaccurate machining parameters, leading to poor quality and potential rework",Verify lathe specifications and drawings before starting the operation
1,Preparation,Unclean or obstructed work area,"Hazardous working conditions, risk of accidents, and potential damage to equipment or materials",Ensure a clean and well-ventilated work area
2,Material Preparation,Insufficient cleaning of the workpiece,"Presence of contaminants on the machined surface, affecting quality and performance",Ensure thorough cleaning of the workpiece before machining
3,Material Preparation,Incorrect selection of workholding device,"Misalignment or inadequate holding of the workpiece, leading to poor machining results",Verify the correct selection of workholding devices based on material specifications
4,Setting Up the Lathe Machine,Improper installation of cutting tool,"Inaccurate cutting parameters, leading to poor quality and potential rework",Ensure proper installation and tightening of cutting tools


In [ ]:
paths = ["./data/Milling_Process.docx"]
text = load_data(paths[0])

big_df, time_ = main(paths)
columns = ["Process Step", "Potential Failure Mode", "Potential Effect", "Recommended Actions"]
big_df = big_df[columns]

big_df.reset_index()
big_df.to_csv("./output_Milling_Process_predict_0420.csv", index = False)
p = save_to_xlsx(big_df, "./output_Milling_Process_predict_0420.xlsx")
print(f"saved to {p}")

big_df
    

In [ ]:
paths = ["./data/Resistance_Welding.docx"]
text = load_data(paths[0])

big_df, time_ = main(paths)
columns = ["Process Step", "Potential Failure Mode", "Potential Effect", "Recommended Actions"]
big_df = big_df[columns]

big_df.reset_index()
big_df.to_csv("./output_Resistance_Welding_predict_0420.csv", index = False)
p = save_to_xlsx(big_df, "output_Resistance_Welding_predict_0420.xlsx")
print(f"saved to {p}")

big_df
    # total took: 374.5574140548706 seconds